# 使用聚类进行特征工程与预处理

**核心概念**: 聚类不仅可以作为最终目标，还可以作为特征工程和预处理的工具，用于降维、特征变换和分类器性能提升

## 应用场景

1. **特征降维**: 用簇标签或到质心距离替代原始特征
2. **半监督学习**: 利用聚类结构传播标签
3. **分类器预处理**: 通过聚类转换提升分类性能
4. **特征增强**: 将聚类特征作为额外输入

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits, load_iris, make_blobs
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# 设置随机种子
np.random.seed(42)

# 配置 matplotlib
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 数据准备

使用鸢尾花数据集演示聚类作为预处理的效果。

In [ ]:
# 加载鸢尾花数据集
iris = load_iris()
X = iris.data
y = iris.target

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")
print(f"特征名称: {iris.feature_names}")
print(f"类别数: {len(np.unique(y))}")

## 2. 基线模型: 直接使用逻辑回归

In [ ]:
# 基线: 直接使用逻辑回归
log_reg_baseline = LogisticRegression(max_iter=1000, random_state=42)
log_reg_baseline.fit(X_train, y_train)

baseline_train_score = log_reg_baseline.score(X_train, y_train)
baseline_test_score = log_reg_baseline.score(X_test, y_test)

print("基线模型 (直接逻辑回归):")
print(f"  训练集准确率: {baseline_train_score:.4f}")
print(f"  测试集准确率: {baseline_test_score:.4f}")

## 3. 聚类特征变换

### 方法 1: 用到质心的距离作为特征

K-Means 的 `transform()` 方法返回每个样本到各个质心的距离，这些距离可以作为新的特征表示。

In [ ]:
# 使用 K-Means 进行特征变换
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
kmeans.fit(X_train)

# 变换: 原始特征 -> 到各质心的距离
X_train_dist = kmeans.transform(X_train)
X_test_dist = kmeans.transform(X_test)

print(f"原始特征维度: {X_train.shape[1]}")
print(f"变换后特征维度: {X_train_dist.shape[1]}")

# 使用变换后的特征训练逻辑回归
log_reg_dist = LogisticRegression(max_iter=1000, random_state=42)
log_reg_dist.fit(X_train_dist, y_train)

dist_train_score = log_reg_dist.score(X_train_dist, y_train)
dist_test_score = log_reg_dist.score(X_test_dist, y_test)

print(f"\n距离特征模型 (K={n_clusters}):")
print(f"  训练集准确率: {dist_train_score:.4f}")
print(f"  测试集准确率: {dist_test_score:.4f}")

## 4. 构建聚类预处理流水线

使用 sklearn Pipeline 将聚类变换和分类器组合起来。

In [ ]:
# 构建流水线: K-Means 特征变换 + 逻辑回归
pipeline = Pipeline([
    ('scaler', StandardScaler()),           # 标准化
    ('kmeans', KMeans(n_clusters=50, n_init=10, random_state=42)),  # 聚类变换
    ('log_reg', LogisticRegression(max_iter=1000, random_state=42)) # 分类器
])

# 训练流水线
pipeline.fit(X_train, y_train)

pipeline_train_score = pipeline.score(X_train, y_train)
pipeline_test_score = pipeline.score(X_test, y_test)

print("流水线模型 (StandardScaler + KMeans + LogisticRegression):")
print(f"  训练集准确率: {pipeline_train_score:.4f}")
print(f"  测试集准确率: {pipeline_test_score:.4f}")

## 5. 使用网格搜索优化聚类数

通过交叉验证找到最佳的聚类数量。

**注意**: Pipeline 中参数命名使用双下划线 `__` 连接步骤名和参数名，如 `kmeans__n_clusters`。

In [ ]:
# 定义参数网格
param_grid = {
    'kmeans__n_clusters': range(5, 51, 5)  # 测试 5, 10, 15, ..., 50
}

# 网格搜索
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\n最佳参数: {grid_search.best_params_}")
print(f"最佳交叉验证分数: {grid_search.best_score_:.4f}")
print(f"测试集准确率: {grid_search.score(X_test, y_test):.4f}")

In [ ]:
# 可视化网格搜索结果
results = grid_search.cv_results_
n_clusters_range = param_grid['kmeans__n_clusters']
mean_scores = results['mean_test_score']
std_scores = results['std_test_score']

plt.figure(figsize=(10, 5))
plt.plot(n_clusters_range, mean_scores, 'bo-', linewidth=2, markersize=8)
plt.fill_between(n_clusters_range, 
                 mean_scores - std_scores, 
                 mean_scores + std_scores, 
                 alpha=0.2)
plt.axvline(x=grid_search.best_params_['kmeans__n_clusters'], 
            color='r', linestyle='--', label=f"最佳 K={grid_search.best_params_['kmeans__n_clusters']}")
plt.xlabel('聚类数量 K', fontsize=12)
plt.ylabel('交叉验证准确率', fontsize=12)
plt.title('聚类数量对分类性能的影响', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. 方法对比

In [ ]:
# 对比不同方法的性能
methods = {
    '基线 (直接逻辑回归)': baseline_test_score,
    f'距离特征 (K={n_clusters})': dist_test_score,
    '流水线 (K=50)': pipeline_test_score,
    f'优化后 (K={grid_search.best_params_["kmeans__n_clusters"]})': grid_search.score(X_test, y_test)
}

print("各方法测试集准确率对比:")
print("-" * 50)
for method, score in methods.items():
    print(f"{method:40s}: {score:.4f}")

# 可视化
plt.figure(figsize=(10, 5))
bars = plt.bar(methods.keys(), methods.values(), color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.ylabel('测试集准确率', fontsize=12)
plt.title('不同预处理方法的分类性能对比', fontsize=14)
plt.ylim(0.9, 1.02)
plt.xticks(rotation=15, ha='right')

# 添加数值标签
for bar, score in zip(bars, methods.values()):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 7. 更复杂的示例: 手写数字识别

在手写数字数据集上展示聚类预处理的效果。

In [ ]:
# 加载手写数字数据集
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"数据集形状: {X_digits.shape}")
print(f"特征数 (像素): {X_digits.shape[1]}")
print(f"类别数: {len(np.unique(y_digits))}")

# 划分数据
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_digits, y_digits, test_size=0.2, random_state=42, stratify=y_digits
)

In [ ]:
# 基线模型
log_reg_digits_baseline = LogisticRegression(max_iter=5000, random_state=42)
log_reg_digits_baseline.fit(X_train_d, y_train_d)

baseline_digits_score = log_reg_digits_baseline.score(X_test_d, y_test_d)
print(f"基线模型准确率: {baseline_digits_score:.4f}")

In [ ]:
# 聚类预处理流水线
pipeline_digits = Pipeline([
    ('scaler', StandardScaler()),
    ('kmeans', KMeans(n_init=10, random_state=42)),
    ('log_reg', LogisticRegression(max_iter=5000, random_state=42))
])

# 快速网格搜索 (简化参数范围以加速)
param_grid_digits = {
    'kmeans__n_clusters': [50, 100, 150, 200]
}

grid_digits = GridSearchCV(
    pipeline_digits, 
    param_grid_digits, 
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_digits.fit(X_train_d, y_train_d)

print(f"\n最佳参数: {grid_digits.best_params_}")
print(f"最佳交叉验证分数: {grid_digits.best_score_:.4f}")
print(f"测试集准确率: {grid_digits.score(X_test_d, y_test_d):.4f}")

In [ ]:
# 手写数字数据集结果对比
print("\n手写数字数据集结果:")
print("-" * 50)
print(f"基线 (直接逻辑回归, 64特征): {baseline_digits_score:.4f}")
print(f"聚类预处理 (K={grid_digits.best_params_['kmeans__n_clusters']}, {grid_digits.best_params_['kmeans__n_clusters']}特征): {grid_digits.score(X_test_d, y_test_d):.4f}")
print(f"\n特征维度变化: 64 -> {grid_digits.best_params_['kmeans__n_clusters']}")

## 8. 特征增强: 组合原始特征和聚类特征

将聚类得到的距离特征与原始特征组合，可能进一步提升性能。

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class KMeansFeatureAugmenter(BaseEstimator, TransformerMixin):
    """
    将 K-Means 距离特征与原始特征组合的自定义转换器
    """
    def __init__(self, n_clusters=10, random_state=42):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.kmeans = None
        
    def fit(self, X, y=None):
        self.kmeans = KMeans(
            n_clusters=self.n_clusters, 
            n_init=10, 
            random_state=self.random_state
        )
        self.kmeans.fit(X)
        return self
    
    def transform(self, X):
        # 计算到各质心的距离
        distances = self.kmeans.transform(X)
        # 组合原始特征和距离特征
        return np.hstack([X, distances])

# 测试特征增强
augmenter = KMeansFeatureAugmenter(n_clusters=20)
X_train_aug = augmenter.fit_transform(X_train)
X_test_aug = augmenter.transform(X_test)

print(f"原始特征维度: {X_train.shape[1]}")
print(f"增强后特征维度: {X_train_aug.shape[1]}")

# 训练分类器
log_reg_aug = LogisticRegression(max_iter=1000, random_state=42)
log_reg_aug.fit(X_train_aug, y_train)

aug_test_score = log_reg_aug.score(X_test_aug, y_test)
print(f"\n特征增强模型准确率: {aug_test_score:.4f}")

## 总结

### 聚类作为预处理的优点

1. **非线性特征变换**: 到质心的距离可以捕获非线性结构
2. **降维**: 用少量质心表示高维数据
3. **特征增强**: 为分类器提供额外的结构信息
4. **可解释性**: 质心具有物理意义

### 使用建议

| 场景 | 建议 |
|------|------|
| 高维数据 | 使用聚类降维 (K < 原始维度) |
| 数据量大 | 考虑 Mini-Batch K-Means |
| 分类性能不足 | 尝试特征增强 |
| 参数选择 | 使用网格搜索交叉验证 |

### Pipeline 参数语法

在 sklearn Pipeline 中，使用双下划线 `__` 访问嵌套组件的参数:

```python
# 格式: 步骤名__参数名
param_grid = {
    'kmeans__n_clusters': [10, 50, 100],
    'log_reg__C': [0.1, 1.0, 10.0]
}
```

### 关键要点

- 聚类预处理是一种无监督特征工程方法
- 最佳聚类数需要通过交叉验证确定
- 特征增强 (原始+聚类) 有时比单独使用效果更好
- 始终使用流水线确保正确的数据泄露预防